In [ ]:
import os
import sys

sys.path.append('../..')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from ase.build import make_supercell
from ase import Atoms
from ase.optimize import BFGS, LBFGS
from HybridFF import HybridFF
from Utilities import show_results, write_xyz, validate
from Constants import EV_TO_KJ, H_TO_KJ, BOHR_TO_ANGSTROM

In [ ]:
DATA = np.load('../../../data/test_sets/DATA_3rd_vanEijck.npy', allow_pickle=True).item()
hybrid_ff = HybridFF(debug=True, pol0=False)

In [ ]:
for SYSTEM in ['VIII', 'X', 'XI']: 
    RESULTS = {}
    MIN_STRUCTURES = {}
    for poly_key in DATA[SYSTEM]:
        try:            
            print(poly_key)
            mol_size = DATA[SYSTEM][poly_key]['mol_size']
            elements_uc = DATA[SYSTEM][poly_key]['uc_symbols']
            coords_uc = DATA[SYSTEM][poly_key]['coords'][None]   
            cells = DATA[SYSTEM][poly_key]['cell'][None]   
            
            atoms = hybrid_ff.initialize(elements_uc, coords_uc, cells, mol_size)    
            optimizer = LBFGS(atoms) # BFGS
            converged = optimizer.run(fmax=1 / EV_TO_KJ, steps=250)
            if converged:
                RESULTS[poly_key] = {}
                MIN_STRUCTURES[poly_key] = {}
                RESULTS[poly_key]['total'] = atoms.get_potential_energy() * EV_TO_KJ / hybrid_ff._n_molecules_uc
                MIN_STRUCTURES[poly_key]['coordinates'] = atoms.get_positions()[None]    
                MIN_STRUCTURES[poly_key]['cell'] = cells
                MIN_STRUCTURES[poly_key]['uc_symbols'] = elements_uc
            np.save(f'data/STRUCTURES_{SYSTEM}.npy', MIN_STRUCTURES)
            np.save(f'data/RESULTS_{SYSTEM}.npy', RESULTS)
        except:
            del RESULTS[poly_key]
            del MIN_STRUCTURES[poly_key]
      